In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from sklearn.cluster import DBSCAN
# from haversine import haversine as hs
import requests
%matplotlib inline
# from pydantic_settings import BaseSettings
#from ydata_profiling import ProfileReport

import sys
import os
# see all columns
pd.set_option('display.max_columns', None)
sys.path.append('../utils/')
#sys.path.append('..\\utils\\')
from functions import *

In [4]:
short_trips = pd.read_csv("../data/processed/sn_shorttrips_accessibility.csv", parse_dates=["ReservationTime", "StartTime", "EndTime", 'StartTimeUpdated'])
short_trips.head()

,BilID,PersonID,LatitudeStart,LongitudeStart,LatitudeEnd,LongitudeEnd,ReservationTime,StartTime,EndTime,Age,Gender,BatteryStart,BatteryEnd,KmDriven,FromZoneID,ToZoneID,TripDurationHours,TripDurationMinutes,AvgSpeed,Month,Weekday,StartHour,HaversineDistance,StartTimeUpdated,TotalDurationMin,TripDistanceKm,TotalWalkingTimeMin,TotalTransitTimeMin,Changes,PickupStationProximity,DropoffStationProximity
0,WBY1Z21040V307905,32614,55.670268,12.586064,55.652954,12.544699,2017-08-16 14:40:00,2017-08-16 14:34:00,2017-08-16 14:52:00,32,M,94.0,91.0,5,103142,102851,0.300000,18.0,16.666667,August,Wednesday,14,3.231816,2023-11-15 14:34:00,29.983333,4.02916,17.983333,12.0,0.0,1138.90,121.03
1,WBY1Z21040V308214,44363,55.687024,12.585952,55.726655,12.553914,2017-08-16 14:42:00,2017-08-16 14:38:00,2017-08-16 15:05:00,25,M,48.0,42.0,7,102212,157231,0.450000,27.0,15.555556,August,Wednesday,14,4.843874,2023-11-15 14:38:00,28.016667,6.11791,16.016667,12.0,0.0,878.06,324.95
2,WBY1Z21040V307869,37425,55.630274,12.648864,55.706866,12.587122,2017-07-31 20:31:00,2017-07-31 20:31:00,2017-08-01 09:13:00,30,M,0.0,0.0,18,185203,102341,12.700000,762.0,1.417323,July,Monday,20,9.358422,2023-11-20 20:31:00,39.866667,12.64301,17.600000,20.0,1.0,382.98,346.59
3,WBY1Z21090V308015,18497,55.700745,12.473436,55.630263,12.648955,2017-07-31 21:11:00,2017-07-31 21:04:00,2017-08-01 04:28:00,42,M,0.0,0.0,28,102651,185203,7.400000,444.0,3.783784,July,Monday,21,13.517087,2023-11-20 21:04:00,52.350000,22.12774,12.350000,31.0,0.0,421.77,329.69
4,WBY1Z21060V308019,28029,55.712655,12.561513,55.658484,12.536456,2017-07-31 21:57:00,2017-07-31 21:56:00,2017-08-01 00:04:00,26,M,0.0,0.0,7,102412,102841,2.133333,128.0,3.281250,July,Monday,21,6.226910,2023-11-20 21:56:00,34.966667,10.41530,11.283333,19.0,1.0,417.46,153.96


In [ ]:
# Assuming df is your dataframe and it contains 'Weekday' and 'StartDate' columns
# You would use the function as follows:
#short_trips.loc[:, 'StartTimeUpdated'] = update_start_date_vectorized(short_trips['Weekday'], short_trips['StartTime'])

In [ ]:
#main_concurrent_sn(short_trips, num_itineraries=3)

In [ ]:
short_trips.head()

In [ ]:
# see the descriptive statistics of the last 7 columns
short_trips.iloc[:, -7:].describe()

In [ ]:
# check for nan values in the last 7 columns
short_trips.iloc[:, -7:].isna().sum()

In [ ]:
# save short_trips to csv
#short_trips.to_csv("../data/processed/sn_shorttrips_accessibility.csv", index=False)

In [ ]:
# see row 299002
short_trips.iloc[308945, :]

In [ ]:
def fetch_and_process_itinerary(response):
    try:
        # Extract itineraries and find the fastest one
        itineraries = response['plan']['itineraries']
        fastest_itinerary = min(itineraries, key=lambda x: x['duration'])
        
        
        total_distance = sum(leg['distance'] for leg in fastest_itinerary['legs']) / 1000
        
        

     

        # Construct the details dictionary
        details = {
            "CarDurationMin": fastest_itinerary['duration'] / 60.0,
            "CarDistanceKm": total_distance,
        }
        return details
    except KeyError as e:
        print(f"Error processing itinerary: {e}")
        # Return NaN values if there's an error in the response format
        return {
            "TotalDurationMin": np.nan,
            "TripDistanceKm": np.nan,
        }
    

# Example usage:
# Assuming `result` is the JSON response parsed into a dictionary
fastest_itinerary = fetch_and_process_itinerary(result)
fastest_itinerary


In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
# function to get the fastest itinerary
# function to get the fastest itinerary
def fetch_and_process_fastest_itinerary(response):
    """
    Fetches and processes the fastest itinerary from the response.

    Args:
        response (dict): The response containing itinerary information.

    Returns:
        dict: A dictionary containing the processed details of the fastest itinerary.
            The dictionary has the following keys:
            - "TotalDurationMin": The total duration of the itinerary in minutes.
            - "TripDistanceKm": The total distance of the itinerary in kilometers.
            - "TotalWalkingTimeMin": The total walking time of the itinerary in minutes.
            - "TotalTransitTimeMin": The total transit time of the itinerary in minutes.
            - "Changes": The number of mode changes in the itinerary.
            - "PickupStationProximity": The distance to the pickup station in meters, or 0 if not available.
            - "DropoffStationProximity": The distance to the dropoff station in meters, or 0 if not available.
    """
    try:
        # Extract itineraries and find the fastest one
        itineraries = response['plan']['itineraries']
        fastest_itinerary = min(itineraries, key=lambda x: x['duration'])
        
        
        total_distance = sum(leg['distance'] for leg in fastest_itinerary['legs']) / 1000
        
        

     

        # Construct the details dictionary
        details = {
            "CarDurationMin": fastest_itinerary['duration'] / 60.0,
            "CarDistanceKm": total_distance,
        }
        return details
    except KeyError as e:
        print(f"Error processing itinerary: {e}")
        # Return NaN values if there's an error in the response format
        return {
            "CarDurationMin": np.nan,
            "CarDistanceKm": np.nan,
        }
    



# This function is a wrapper around the 'fetch_and_process_fastest_itinerary' for concurrent processing
def fetch_and_process_itinerary_concurrent(start_lat, start_lon, end_lat, end_lon, start_time, mode, num_itineraries):
    response = get_top_itineraries(start_lat, start_lon, end_lat, end_lon, start_time, mode=mode, num_itineraries=num_itineraries)
    return fetch_and_process_fastest_itinerary(response)

# The main function to process the DataFrame concurrently and update the results
def main_concurrent_sn(df, num_itineraries, mode):
    """
    Process itineraries concurrently using a thread pool executor.

    Args:
        df (pandas.DataFrame): The input DataFrame containing itinerary data.
        num_itineraries (int): The number of itineraries to fetch and process for each row.

    Returns:
        None
    """
    # Temporary dictionary to hold the results
    results = {}
    with ThreadPoolExecutor(max_workers=50) as executor:
        # Prepare and submit all futures
        futures_to_index = {
            executor.submit(
                fetch_and_process_itinerary_concurrent,
                row['LatitudeStart'], row['LongitudeStart'],
                row['LatitudeEnd'], row['LongitudeEnd'],
                row['StartTimeUpdated'], mode,
                num_itineraries
            ): index for index, row in df.iterrows()
        }
        
        # Process futures as they complete and show progress
        for future in tqdm(as_completed(futures_to_index), total=len(futures_to_index), desc="Processing itineraries"):
            index = futures_to_index[future]
            try:
                results[index] = future.result()
            except Exception as exc:
                print(f'Row {index} generated an exception: {exc}')
                results[index] = {
                    "CarDurationMin": np.nan,
                    "CarDistanceKm": np.nan,
                }
    
    # Update the DataFrame outside the thread pool
    for index, data in results.items():
        for key, value in data.items():
            df.at[index, key] = value

In [ ]:
#main_concurrent_sn(short_trips, num_itineraries=2, mode = "CAR_PICKUP")

In [ ]:
# see all the columns with pd
pd.set_option('display.max_columns', None)
short_trips.head()

In [ ]:
# drop CarDurationMin
short_trips = short_trips.drop(columns=["CarDurationMin"])

# rename CaarDurationMin to CarDurationMin
short_trips = short_trips.rename(columns={"CaarDurationMin": "CarDurationMin"})

In [ ]:
short_trips.head()

In [ ]:
short_trips.CarDistanceKm.isna().sum()

In [ ]:
# return the descriptive statistics of HaversineDistance for the rows where CarDistanceKm is null
short_trips.loc[short_trips.CarDistanceKm.isna(), 'HaversineDistance'].describe()

In [ ]:
# plot the KmDriven against CarDistanceKm
# set figure size
plt.figure(figsize=(10, 8))
sns.scatterplot(data=short_trips, x="HaversineDistance", y="CarDistanceKm", alpha=0.9)
plt.title("KmDriven vs CarDistanceKm")
sns.despine()
plt.show()

In [ ]:
# save short_trips to csv
#short_trips.to_csv("../data/processed/sn_shtrips_access_complete.csv", index=False)
